In [1]:
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn

from configs.CC import Config
from peleenet import build_net

from utils.core import anchors, init_net, get_dataloader,\
print_info, image_forward, nms_process
from utils.timer import Timer

from layers.functions import Detect, PriorBox
from data import BaseTransform

from tqdm import tqdm
import pickle

## Build net

In [2]:
## Build net

cfgfile = "./configs/Pelee_VOC.py"

cfg = Config.fromfile(cfgfile)
net = build_net('test', cfg.model.input_size, cfg.model)

path2pretrained = "/workspace/raid/data/jgusak/Pelee.Pytorch/Pelee_VOC.pth"
init_net(net, cfg, path2pretrained)

net.eval()

Loading resume network...


PeleeNet(
  (features): Sequential(
    (stemblock): _StemBlock(
      (stem1): conv_bn_relu(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (stem2a): conv_bn_relu(
        (conv): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (stem2b): conv_bn_relu(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (stem3): conv_bn_relu(
        (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0

## Compress net (layers from net.features, for example)

    -Noting to compress inside net.loc, net.conf, net.softmax \
    -one 3x3 conv in each of 5 ResBlock in net.resblock\
    -one 3x3 conv in net.extras.1, net.extras.3, net.extras.5\ 
    -many 3x3 branch1b.conv, branch2b.conv, branch2c.conv in net.features

In [3]:
import copy
import sys
sys.path.append('/workspace/home/jgusak/musco/')

from model_utils import get_layer_names
from tensor_compression import get_compressed_model
from flopco import FlopCo

Using numpy backend.
Using pytorch backend.


In [9]:
layer_names, conv_layer_mask = get_layer_names(net)

fc_layer_mask = (1 - conv_layer_mask).astype(bool)

print(layer_names[conv_layer_mask])
# print(layer_names[fc_layer_mask])

['features.stemblock.stem1.conv' 'features.stemblock.stem2a.conv'
 'features.stemblock.stem2b.conv' 'features.stemblock.stem3.conv'
 'features.denseblock1.denselayer1.branch1a.conv'
 'features.denseblock1.denselayer1.branch1b.conv'
 'features.denseblock1.denselayer1.branch2a.conv'
 'features.denseblock1.denselayer1.branch2b.conv'
 'features.denseblock1.denselayer1.branch2c.conv'
 'features.denseblock1.denselayer2.branch1a.conv'
 'features.denseblock1.denselayer2.branch1b.conv'
 'features.denseblock1.denselayer2.branch2a.conv'
 'features.denseblock1.denselayer2.branch2b.conv'
 'features.denseblock1.denselayer2.branch2c.conv'
 'features.denseblock1.denselayer3.branch1a.conv'
 'features.denseblock1.denselayer3.branch1b.conv'
 'features.denseblock1.denselayer3.branch2a.conv'
 'features.denseblock1.denselayer3.branch2b.conv'
 'features.denseblock1.denselayer3.branch2c.conv'
 'features.transition1.conv'
 'features.denseblock2.denselayer1.branch1a.conv'
 'features.denseblock2.denselayer1.bran

In [8]:
model = copy.deepcopy(net)
compressed_model = get_compressed_model(model,
                                        ranks=[0],
                                        layer_names=['features.denseblock1.denselayer2.branch2b.conv'],
                                        decompositions = ['tucker2'],
                                        vbmf_weaken_factor=1.0)
compressed_model.to('cuda')

Decompose layer features.denseblock1.denselayer2.branch2b.conv
	 new rank:  [16, 16]


PeleeNet(
  (features): Sequential(
    (stemblock): _StemBlock(
      (stem1): conv_bn_relu(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (stem2a): conv_bn_relu(
        (conv): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (stem2b): conv_bn_relu(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (stem3): conv_bn_relu(
        (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0

In [11]:
flopco_m = FlopCo(net.features)
flopco_m.total_flops

1027096448

In [12]:
flopco_cm = FlopCo(compressed_model.features)
flopco_cm.total_flops

1030307712

## Save compressed net

In [13]:
torch.save(compressed_model.state_dict(), '/workspace/raid/data/jgusak/Pelee.Pytorch/Pelee_VOC_compressed.pth')

## Draft

In [7]:
from torchvision.models import resnet18, resnet50
model2 = resnet50()

flopco_m2 = FlopCo(model2)

In [8]:
flopco_m2.total_flops

8234241536

In [9]:
flopco_m2.total_flops/flopco_m.total_flops

8.017009066708408

In [20]:
len(model2.layer1), len(model2.layer2), len(model2.layer3), len(model2.layer4)

(3, 4, 6, 3)

In [21]:
dict(model2.named_children()).keys()

dict_keys(['conv1', 'bn1', 'relu', 'maxpool', 'layer1', 'layer2', 'layer3', 'layer4', 'avgpool', 'fc'])

In [22]:
model2

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [12]:
p = "/workspace/raid/data/jgusak/Pelee.Pytorch/weights/peleenet.pth"

mo = torch.load(p)
type(mo)

collections.OrderedDict